### Quiz 3

#### Submitted by - Fawad Kirmani

Modified on - Oct 19 2020

cite: https://spacy.io/

cite: https://spacy.io/usage/rule-based-matching

In [1]:
import re 
import pandas as pd
import PyPDF2
from itertools import cycle
import warnings
warnings.filterwarnings('ignore')

Saved the docx file provided by professor as txt file with utf-8 encoding before importing.

In [2]:
inupt_file = open('./common-data/dataset-nyt-nobel2020_ver2.txt','r+')  

Reading the txt file.

In [3]:
file_content = inupt_file.read()
file_content

'https://www.nytimes.com/article/2020-nobel-prize-winners.html \n2020 Nobel Prize Winners: Full List\nNobel Prize season begins every October as committees in Sweden and Norway name laureates in a variety of prizes in the sciences, literature and economics, as well as peace work. The announcements started last week with the awarding of the prize in Physiology or Medicine. They wrapped up on Monday, when the Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel was announced.\nThe Nobel Prizes most years are presented to recipients in Stockholm and Oslo in December. Because of the coronavirus pandemic, the committees are changing their approaches. Some of the events in Stockholm will be canceled in favor of a digital ceremony for the Nobelists, and medals and diplomas are to be distributed to the recipients’ embassies and handed over in their home countries. Recipients may be invited to the award ceremony for 2021, if possible.\nThe Oslo ceremony for the peace prize wil

#### Running simple entity detection using spacy

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
doc = nlp(file_content)
for ent in doc.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char)

Nobel Prize Winners WORK_OF_ART 68 87
October DATE 131 138
Sweden GPE 156 162
Norway GPE 167 173
last week DATE 304 313
Physiology or Medicine WORK_OF_ART 348 370
Monday DATE 391 397
Alfred Nobel PERSON 466 478
The Nobel Prizes WORK_OF_ART 494 510
most years DATE 511 521
Stockholm GPE 553 562
Oslo GPE 567 571
December DATE 575 583
Stockholm GPE 690 699
Nobelists NORP 756 765
2021 DATE 935 939
Oslo GPE 958 962
most years DATE 1016 1026
Nobel WORK_OF_ART 1057 1062
last month DATE 1103 1113
10 million CARDINAL 1139 1149
Swedish NORP 1150 1157
1 million CARDINAL 1165 1174
the previous year DATE 1188 1205
about $112,000 MONEY 1243 1257
2020 DATE 1289 1293
Harvey J. Alter PERSON 1330 1345
Michael Houghton PERSON 1347 1363
Charles M. Rice PERSON 1368 1383
Monday DATE 1387 1393
Nobel WORK_OF_ART 1463 1468
three CARDINAL 1488 1493
millions CARDINAL 1570 1578
Roger Penrose PERSON 1599 1612
Reinhard Genzel PERSON 1614 1629
Andrea Ghez PERSON 1634 1645
the Nobel Prize in Physics WORK_OF_ART 1668 1

In above spacy entity detection, we are not seeing prize categories except for Physiology or Medicine and Chemistry. Also, there labels are different. As we already know the Nobel prize categories, we can use that and filter the list of winners and event day accordingly.

Now, I will categorize all the prize categories in to one label and use that to detect their location in the document. This will help me is extracting winners for each category and day of announcement of each prize category. Labeling these prize categories as PRIZE_CATEGORIES

Also, reconizing positions where "received" and "awarded" is used in sentences to declare prize winners. Labeling them as AWARD_RECEIVED

In [6]:
from spacy.pipeline import EntityRuler

nlp = spacy.load('en_core_web_sm')

# weights_pattern1 = [
#     {"TEXT": {"IN": ['Physics', 'Chemistry', 'Literature', 'Peace']}}
# ]
# weights_pattern2 = [
#     {"ORTH": {"IN": ['received', 'awarded']}}
# ]

# patterns = [{"label": "PRIZE_CATEGORIES", "pattern": weights_pattern1}, 
#             {"label": "AWARD_RECEIVED", "pattern": weights_pattern2}]
# ruler = EntityRuler(nlp, patterns=patterns)
# nlp.add_pipe(ruler, before="ner")

ruler = EntityRuler(nlp, overwrite_ents=True, validate=True)
ruler.add_patterns([
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "economic"}, {"TEXT": "science"}]},
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "Physiology"}, {"TEXT": "or"}, {"TEXT": "Medicine"}]},
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "Physics"}]},
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "Chemistry"}]},
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "Literature"}]},
    {"label": "PRIZE_CATEGORIES", "pattern": [{"TEXT": "Peace"}]},
    {"label": "PRIZE_CATEGORIES_REPEAT", "pattern": [{"TEXT": "Economic"}, {"TEXT": "Science"}]},
    {"label": "AWARD_RECEIVED", "pattern": [{"TEXT": "received"}]},
    {"label": "AWARD_RECEIVED", "pattern": [{"TEXT": "awarded"}]},
])
nlp.add_pipe(ruler)

doc = nlp(file_content)

txt = []
label = []
start_char = []
end_char = []
with doc.retokenize() as retokenizer:
    for ent in doc.ents:
        retokenizer.merge(doc[ent.start:ent.end])
        txt.append(ent.text)
        label.append(ent.label_)
        start_char.append(ent.start_char)
        end_char.append(ent.end_char)
        print(ent.text, ent.label_, ent.start_char, ent.end_char)
data_label_list = list(zip(txt, label, start_char, end_char))
data_label_table = pd.DataFrame(data_label_list, columns=['txt', 'label', 'start_char', 'end_char'])

Nobel Prize Winners WORK_OF_ART 68 87
October DATE 131 138
Sweden GPE 156 162
Norway GPE 167 173
last week DATE 304 313
Physiology or Medicine PRIZE_CATEGORIES 348 370
Monday DATE 391 397
Alfred Nobel PERSON 466 478
The Nobel Prizes WORK_OF_ART 494 510
most years DATE 511 521
Stockholm GPE 553 562
Oslo GPE 567 571
December DATE 575 583
Stockholm GPE 690 699
Nobelists NORP 756 765
2021 DATE 935 939
Oslo GPE 958 962
most years DATE 1016 1026
Nobel WORK_OF_ART 1057 1062
last month DATE 1103 1113
10 million CARDINAL 1139 1149
Swedish NORP 1150 1157
1 million CARDINAL 1165 1174
the previous year DATE 1188 1205
about $112,000 MONEY 1243 1257
2020 DATE 1289 1293
Physiology or Medicine PRIZE_CATEGORIES 1302 1324
Harvey J. Alter PERSON 1330 1345
Michael Houghton PERSON 1347 1363
Charles M. Rice PERSON 1368 1383
Monday DATE 1387 1393
received AWARD_RECEIVED 1394 1402
Nobel WORK_OF_ART 1463 1468
three CARDINAL 1488 1493
millions CARDINAL 1570 1578
Physics PRIZE_CATEGORIES 1590 1597
Roger Penrose 

#### Filtering the rerquired data using custom labels created in the above steps.

Counting how many times received and awarded is used in the document.

In [7]:
received_count = len(data_label_table[data_label_table['txt']=='received']['txt'])
awarded_count = len(data_label_table[data_label_table['txt']=='awarded']['txt'])

Creating winner, event day and prize categories list. As historically, nobel winners have been individuals and organizations, including them both in the winners list.

In [8]:
winner_list = []
prize_category = []
events = []
for i in range(0, received_count):
    pos = data_label_table[data_label_table['txt']=='received'].index[i]
    search = range(pos - (7-1), pos + 2)
    winners1 = list(data_label_table.loc[search][(data_label_table.loc[search]['label']=='PERSON') | (data_label_table.loc[search]['label']=='ORG')]['txt'])[-3:]
    winner_list.append(winners1)
    prize_category1 = list(data_label_table.loc[search][data_label_table.loc[search]['label']=='PRIZE_CATEGORIES']['txt'])
    prize_category.append(list(set(prize_category1)))
    time1 = list(data_label_table.loc[search][data_label_table.loc[search]['label']=='DATE']['txt'])[-1:]
    events.append(list(set(time1)))
    print("<", ', '.join(winners1), 
          "recieved Nobel prize in",
          (prize_category1)[0], 
          "on", 
          ','.join(time1),">")
#     print(data_label_table.loc[search])
for i in range(0, awarded_count):
    pos = data_label_table[data_label_table['txt']=='awarded'].index[i]
    search = range(pos - (3-1), pos + 4)
    winners2 = list(data_label_table.loc[search][(data_label_table.loc[search]['label']=='PERSON') | (data_label_table.loc[search]['label']=='ORG')]['txt'])
    winner_list.append(winners2)
    prize_category2 = list(data_label_table.loc[search][data_label_table.loc[search]['label']=='PRIZE_CATEGORIES']['txt'])
    prize_category.append(list(set(prize_category2)))
    time2 = list(data_label_table.loc[search][data_label_table.loc[search]['label']=='DATE']['txt'])
    events.append(list(set(time2)))
    print("<", ', '.join(winners2), 
          "recieved Nobel prize in",
          (prize_category2)[0], 
          "on", 
          ','.join(time2),">")
#     print(data_label_table.loc[search])

< Harvey J. Alter, Michael Houghton, Charles M. Rice recieved Nobel prize in Physiology or Medicine on Monday >
< Roger Penrose, Reinhard Genzel, Andrea Ghez recieved Nobel prize in Physics on Tuesday >
< Emmanuelle Charpentier, Jennifer A. Doudna recieved Nobel prize in Chemistry on Wednesday >
< Louise Glück recieved Nobel prize in Literature on Thursday >
< the World Food Program recieved Nobel prize in Peace on Friday >
< Paul R. Milgrom, Robert B. Wilson recieved Nobel prize in economic science on Monday >


#### List of Winners

In [9]:
winner_list

[['Harvey J. Alter', 'Michael Houghton', 'Charles M. Rice'],
 ['Roger Penrose', 'Reinhard Genzel', 'Andrea Ghez'],
 ['Emmanuelle Charpentier', 'Jennifer A. Doudna'],
 ['Louise Glück'],
 ['the World Food Program'],
 ['Paul R. Milgrom', 'Robert B. Wilson']]

#### List of Prize categories

In [10]:
prize_category

[['Physiology or Medicine'],
 ['Physics'],
 ['Chemistry'],
 ['Literature'],
 ['Peace'],
 ['economic science']]

#### List of days the prizes were announced

In [11]:
events

[['Monday'], ['Tuesday'], ['Wednesday'], ['Thursday'], ['Friday'], ['Monday']]

#### Creating table with all the extracted entities 

In [12]:
entity_list = list(zip(prize_category, winner_list, events))
entity_table = pd.DataFrame(entity_list, columns=['prize_category', 'winners', 'event'])
entity_table

,prize_category,winners,event
0,[Physiology or Medicine],"[Harvey J. Alter, Michael Houghton, Charles M....",[Monday]
1,[Physics],"[Roger Penrose, Reinhard Genzel, Andrea Ghez]",[Tuesday]
2,[Chemistry],"[Emmanuelle Charpentier, Jennifer A. Doudna]",[Wednesday]
3,[Literature],[Louise Glück],[Thursday]
4,[Peace],[the World Food Program],[Friday]
5,[economic science],"[Paul R. Milgrom, Robert B. Wilson]",[Monday]


Calculating Precision, Recall and F1 scores. This is to make sure I am able to get right information from the txt file.

Created true labels using the text file. 

In [13]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [14]:
y_true_winners = ['Harvey J. Alter', 'Michael Houghton', 'Charles M. Rice',
                  'Roger Penrose', 'Reinhard Genzel', 'Andrea Ghez', 
                  'Emmanuelle Charpentier', 'Jennifer A. Doudna',
                  'Louise Glück',
                  'the World Food Program',
                  'Paul R. Milgrom', 'Robert B. Wilson']
y_pred_winners = []
for l in winner_list:
    for k in l:
        y_pred_winners.append(k)

In [15]:
precision_recall_fscore_support(y_true_winners, y_pred_winners, average='micro')

(1.0, 1.0, 1.0, None)

In [16]:
precision_recall_fscore_support(y_true_winners, y_pred_winners, average='macro')

(1.0, 1.0, 1.0, None)

In [17]:
y_true_cat = ['Physiology or Medicine',
              'Physics',
              'Chemistry',
              'Literature',
              'Peace',
              'economic science']
y_pred_cat = []
for l in prize_category:
    for k in l:
        y_pred_cat.append(k)

In [18]:
precision_recall_fscore_support(y_true_cat, y_pred_cat, average='micro')

(1.0, 1.0, 1.0, None)

In [19]:
precision_recall_fscore_support(y_true_cat, y_pred_cat, average='macro')

(1.0, 1.0, 1.0, None)

In [20]:
y_true_event = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday']
y_pred_event = []
for l in events:
    for k in l:
        y_pred_event.append(k)

In [21]:
precision_recall_fscore_support(y_true_event, y_pred_event, average='micro')

(1.0, 1.0, 1.0, None)

In [22]:
precision_recall_fscore_support(y_true_event, y_pred_event, average='macro')

(1.0, 1.0, 1.0, None)